In [ ]:
#Kütüphane
!pip install pydicom
!pip install pillow
!pip install tensorflow
!pip install scikit-learn
!pip install opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.0 MB/s eta 0:00:00


In [ ]:
import os

data_dir = '/content/drive/MyDrive/Tekno/denemeler'

# Ana dizin içeriğini kontrol et
print("Ana dizin içeriği:")
print(os.listdir(data_dir))

# Sınıf klasörlerinin içeriğini kontrol et
for class_name in ['normal', 'kitle', 'kalsifikasyon']:
    class_dir = os.path.join(data_dir, class_name)
    print(f"{class_name} klasörü içeriği:")
    if os.path.exists(class_dir):
        print(os.listdir(class_dir))
    else:
        print(f"{class_name} klasörü bulunamadı.")


Ana dizin içeriği:
['.ipynb_checkpoints', '11637', '12443', '11374', '12539', '12614', '12682', '12321', '12369', '12411', '12012', '12065', '12137', '11824', '11866', '11894', '11779', '11718', '11372', '11403', '11436', '11078', '11121', '11180', '10082', '10144', '10210', '10267', 'normal', 'LMLO.png', 'LCC.png', 'umarımsonolur']
normal klasörü içeriği:
['LCC.png', 'LMLO.png']
kitle klasörü içeriği:
kitle klasörü bulunamadı.
kalsifikasyon klasörü içeriği:
kalsifikasyon klasörü bulunamadı.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Çalışan KOD
import os
import pydicom
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf
import csv

labels = {}

label_coordinates = []

# DICOM dosyalarının bulunduğu klasör
dcm_folder = '/content/drive/MyDrive/Teknofest-2024/Kategori5Sag/12431'

kategori = 'Kategori5'
kategorinum = '12431'

# Çıktı dosyalarının kaydedileceği klasör
output_folder = '/content/drive/MyDrive/Tekno/denemeler/umarımsonolur/12431'
os.makedirs(output_folder, exist_ok=True)

# Modeli yükleyin (bu modeli kendiniz eğitmeniz veya önceden eğitilmiş bir model kullanmanız gerekebilir)
model_path = '/content/drive/MyDrive/Tekno/11.06-derleme.h5'
model = tf.keras.models.load_model(model_path)

# Görüntü ve etiketler listesi
images_with_labels = []

# DICOM dosyalarını oku ve tahmin et
for root, dirs, files in os.walk(dcm_folder):
    for file_name in files:
        if file_name.endswith('.dcm'):
            file_path = os.path.join(root, file_name)
            dicom_data = pydicom.dcmread(file_path)

            # DICOM verilerini numpy array olarak al
            image_array = dicom_data.pixel_array

            # Görüntüyü normalize et ve yeniden boyutlandır
            image_array = image_array.astype(float)
            image_array = (np.maximum(image_array, 0) / image_array.max()) * 255.0
            image_array = np.uint8(image_array)

            # Görüntüyü RGB formata dönüştür
            input_image_rgb = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)

            # Görüntüyü modelin beklediği boyutlara yeniden boyutlandır
            input_image_resized = cv2.resize(input_image_rgb, (224, 224))
            input_image_resized = np.expand_dims(input_image_resized, axis=0)  # Batch boyutu ekleyin

            # Modeli kullanarak tahmin yap
            prediction = model.predict(input_image_resized)
            label_index = np.argmax(prediction, axis=1)[0]

            # Etiketleri belirle
            if label_index == 0:
                label = 'Normal'
            elif label_index == 1:
                label = 'Kitle'
            elif label_index == 2:
                label = 'Kalsifikasyon'
            else:
                label = 'Bulunamadı'

            # PNG olarak kaydet
            label_folder = os.path.join(output_folder, label)
            os.makedirs(label_folder, exist_ok=True)
            output_file_path = os.path.join(label_folder, file_name.replace('.dcm', '.png'))
            Image.fromarray(image_array).save(output_file_path)

            # Görüntüyü ve etiketi listeye ekle
            images_with_labels.append((output_file_path, label))

print(f"Toplam {len(images_with_labels)} dosya işlendi ve etiketlendi.")

# Görüntüler ve etiketler listesini yazdır
for image_path, label in images_with_labels:
  print(f"Görüntü: {image_path}, Etiket: {label}")

# Görüntüler ve etiketler listesini işleyin ve etiketli görüntüleri kaydedin
for image_path, label in images_with_labels:
    image = cv2.imread(image_path)

    # Sınırlayıcı kutuyu çizin
    if label != 'Bulunamadı':
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 1.50
        fontColor = (48, 48, 48)
        lineType = 2
        thickness = 2
        #print(label)

        # Sınırlayıcı kutunun koordinatlarını alın
        # Etiketin görüntüye eklenmesi
        bottomLeftCornerOfText = (10, 50)
        contours, _ = cv2.findContours(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            #cv2.rectangle(image, (x, y), (x + w, y + h), (48, 255, 48), thickness)
            cv2.putText(image, label, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
            coordinate_string = f"{x},{y};{x},{y + h};{x + w},{y + h};{x + w},{y}"
            label_coordinates.append((file_name, label, coordinate_string))

    # Etiketlenmiş görüntüyü kaydedin
    output_file_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(output_file_path, image)

    labels[file_name] = label

    # Etiketlenmiş görüntüyü kaydet
    output_labeled_path = os.path.join(output_folder, f"labeled_{os.path.basename(image_path)}")
    cv2.imwrite(output_labeled_path, image)

    # Modeli kullanarak tahmin yap
    prediction = model.predict(input_image_resized)[0]

    item = label_coordinates
csv_file_path = '/content/drive/MyDrive/Tekno/labels.csv'
file_exists = os.path.isfile(csv_file_path)

with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:  # 'a' modunda açarak dosyaya ekleme yapıyoruz
    csvwriter = csv.writer(csvfile)
    if not file_exists:
        csvwriter.writerow(["KATEGORİ", "HASTA ID", "DOSYA ADI", "ETİKET ADI", "ETİKET KOORDİNATLARI"])  # Başlıkları yaz
    for item in label_coordinates:
        for file_name, label in labels.items():
          print("Konum Değeleri: " + item[2])  # Değerleri görmek için
    csvwriter.writerow([kategori, kategorinum, file_name, label, item[2]])

print(f"Toplam {len(labels)} dosya işlendi ve etiketlendi.")

1/1 [==============================] - 0s 158ms/step
Toplam 2 dosya işlendi ve etiketlendi.
Görüntü: /content/drive/MyDrive/Tekno/denemeler/umarımsonolur/12431/Kitle/RCC.png, Etiket: Kitle
Görüntü: /content/drive/MyDrive/Tekno/denemeler/umarımsonolur/12431/Kitle/RMLO.png, Etiket: Kitle
1/1 [==============================] - 0s 221ms/step
Konum Değeleri: 0,0;0,2964;2364,2964;2364,0
Konum Değeleri: 0,0;0,2964;2364,2964;2364,0
Toplam 1 dosya işlendi ve etiketlendi.
